# Miscellaneous

## Publishers download

In [4]:
import requests
import time

In [9]:
BASE_URL = "https://api.openalex.org/publishers"

def fetch_page(cursor):
    params = {
            "cursor": cursor,
            "per-page": 200,
        }
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    return r.json()

cursor = "*"
count = 0
output_file = f"../data/interim/openalex_publishers.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    pbar = tqdm(unit="publishers", dynamic_ncols=True)
    while True:
        data = fetch_page(cursor)

        publishers = data.get("results", [])
        next_cursor = data.get("meta", {}).get("next_cursor", None)

        for p in publishers:
            f.write(json.dumps(p) + "\n")
            count += 1

        pbar.update(len(publishers))
            
        if not next_cursor:
            break
        cursor = next_cursor
        time.sleep(1)

    pbar.close()

print("\n=== COMPLETED ===")

0publishers [00:47, ?publishers/s]
10719publishers [01:31, 116.81publishers/s]


=== COMPLETED ===


In [57]:
interest = ['id', 'display_name', 'hierarchy_level', 'parent_publisher']


records = []
with open(f"../data/interim/openalex_publishers.jsonl", "rb") as f:
    for line in f:
        rec = orjson.loads(line)
        filtered = {k: rec.get(k) for k in interest}

        records.append(filtered)

df_publishers = pd.DataFrame(records)
df_publishers['parent_publisher'] = df_publishers['parent_publisher'].apply(lambda x: x['display_name'] if x != None else x)
df_publishers_hierar = df_publishers[df_publishers['hierarchy_level'] == 2][['display_name', 'parent_publisher']].merge(df_publishers, left_on = 'parent_publisher', right_on = 'display_name')\
                                                                                                 .drop(columns = ['parent_publisher_x', 'display_name_y']).rename(columns = {'display_name_x' : 'display_name',\
                                                                                                                  'parent_publisher_y' : 'parent_publisher'})
df_publishers_final = pd.concat((df_publishers[df_publishers['hierarchy_level'] != 2], df_publishers_hierar), ignore_index = True)
df_publishers_final.to_csv('publishers.csv')
df_publishers_final

,id,display_name,hierarchy_level,parent_publisher
0,https://openalex.org/P4310320990,Elsevier BV,0.0,None
1,https://openalex.org/P4310320595,Wiley,0.0,None
2,https://openalex.org/P4310319900,Springer Science+Business Media,1.0,Springer Nature
3,https://openalex.org/P4310311648,Oxford University Press,1.0,University of Oxford
4,https://openalex.org/P4310320547,Taylor & Francis,0.0,None
...,...,...,...,...
10714,https://openalex.org/P4310315789,UCLA Department of Applied Linguistics,1.0,University of California
10715,https://openalex.org/P4310312461,Programa de Pós-graduação em Artes Cênicas (USP),1.0,University of São Paulo
10716,https://openalex.org/P4310315789,UCLA Department of Urban Planning,1.0,University of California
10717,https://openalex.org/P4310312461,Departamento de Comunicações e Artes da Univer...,1.0,University of São Paulo
